determine which icd 10 need conversion

convert with mpa

try leftover with remaining

In [1]:
import pandas as pd

## Read rare disease information

Read rare disease information and filter down just to rare diseases with ICD-10 codes.

In [2]:
dises = (pd
    .read_csv("data/rare_disease_info.tsv", sep='\t')
    .query("ref_name == 'ICD-10'")
    [["dise_name", "orphanet_id", "ref_id"]]
    .rename(columns={"ref_id": "icd_id"})
)

In [3]:
dises.shape

(8266, 3)

In [4]:
dises.head(2)

,dise_name,orphanet_id,icd_id
0,"Multiple epiphyseal dysplasia, Al-Gazali type",166024,Q77.3
2,"Multiple epiphyseal dysplasia, with miniepiphyses",166032,Q77.3


## Read SNOMED/ICD-10 mapping information

In [5]:
code_map = pd.read_csv("data/snomed_icd10_map.tsv", sep='\t')

In [6]:
code_map.head()

,snomed_id,snomed_name,icd_id,icd_name
0,109006,Anxiety disorder of childhood OR adolescence (...,F93.0,Separation anxiety disorder of childhood
1,109006,Anxiety disorder of childhood OR adolescence (...,F40.8,Other phobic anxiety disorders
2,109006,Anxiety disorder of childhood OR adolescence (...,F94.8,Other childhood disorders of social functioning
3,109006,Anxiety disorder of childhood OR adolescence (...,F40.10,"Social phobia, unspecified"
4,109006,Anxiety disorder of childhood OR adolescence (...,F93.8,Other childhood emotional disorders


## Convert Orphanet diseases to SNOMED id space

In [7]:
dises = dises.merge(code_map[["snomed_id", "icd_id"]], how="left", on="icd_id")

In [8]:
dises.shape

(156031, 4)

In [9]:
dises.head()

,dise_name,orphanet_id,icd_id,snomed_id
0,"Multiple epiphyseal dysplasia, Al-Gazali type",166024,Q77.3,76556008.0
1,"Multiple epiphyseal dysplasia, Al-Gazali type",166024,Q77.3,205465004.0
2,"Multiple epiphyseal dysplasia, Al-Gazali type",166024,Q77.3,254082007.0
3,"Multiple epiphyseal dysplasia, Al-Gazali type",166024,Q77.3,254083002.0
4,"Multiple epiphyseal dysplasia, Al-Gazali type",166024,Q77.3,278715001.0


In [10]:
dises.isnull().sum()

dise_name         0
orphanet_id       0
icd_id            0
snomed_id      2640
dtype: int64

Some of the Orphanet rare diseases don't have SNOMED ids. We will try to map these diseases directly to SNOMED through exact disease name matching.

In [19]:
missing.head()

,dise_name,orphanet_id,icd_id,snomed_id
24,Alexander disease,58,E75.2,NaN
71,Multiple sulfatase deficiency,585,E75.2,NaN
89,Canavan disease,141,E75.2,NaN
225,Cystinosis,213,E72.0,NaN
311,Farber disease,333,E75.2,NaN


In [20]:
missing[["dise_name", "orphanet_id"]].drop_duplicates().shape

(2357, 2)

In [11]:
missing = dises[dises["snomed_id"].isnull()]

In [12]:
miss = set(missing["orphanet_id"])

In [13]:
a = pd.read_csv("data/rare_disease_info.tsv", sep='\t')

In [14]:
a.head(2)

,dise_name,orphanet_id,ref_id,ref_name,dise_type
0,"Multiple epiphyseal dysplasia, Al-Gazali type",166024,Q77.3,ICD-10,rare_genetic
1,"Multiple epiphyseal dysplasia, Al-Gazali type",166024,607131,OMIM,rare_genetic


In [15]:
omim = set(a.query("ref_name == 'OMIM'")["orphanet_id"])
umls = set(a.query("ref_name == 'UMLS'")["orphanet_id"])

In [16]:
len(omim & miss)

1330

In [17]:
len(umls & miss)

858

In [18]:
len(miss - umls - omim)

735